In [1]:
# you get all pretrained model name here
# https://huggingface.co/transformers/pretrained_models.html
import pandas as pd
import os
import torch
import transformers
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
# from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [2]:
len(os.listdir('models-vbert'))

0

In [3]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
# model_name = "bert-base-uncased"
model_name = "vBERT-2020-Base-PyTorch"
model_dir = 'models-vbert'
# max sequence length for each document/sentence sample
max_length = 512

In [4]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
# tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/api/models/vBERT-2020-Base-PyTorch

In [ ]:
df = pd.read_csv('data/importance_detection_30_12000_rand0.csv')

In [ ]:
df.head()

In [ ]:
def read_passages(path_data, test_size=0.2):
    df = pd.read_csv(path_data)
    documents = df['log'].to_list()
    labels_str = df['pcc'].to_list()
    labels_list = sorted(list(set(labels_str)))
    labels_all = {l:idx for idx, l in enumerate(labels_list)}
    labels = [labels_all[label_str] for label_str in labels_str]
    return train_test_split(documents, labels, test_size=test_size), labels_list
# end

In [ ]:
# call the function
dir_data = 'data'
name_data_file = 'importance_detection_30_12000_rand0.csv'
path_data_relative = os.path.join(dir_data, name_data_file)
(train_texts, valid_texts, train_labels, valid_labels), target_names = read_passages(path_data_relative)

In [ ]:
train_labels[0]

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

In [ ]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

In [ ]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
if len(os.listdir('models')) > 0:
    print('load model from local')
    model_info = model_dir
else:
    print('load model from official')
    model_info = model_name
    
model = BertForSequenceClassification.from_pretrained(model_info)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    # pred = np.argmax(pred, axis=1)
    preds = pred.predictions.argmax(-1).reshape(-1)

#     print('labels: {}'.format(labels))
#     print('pred: {}'.format(preds))
    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1)
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1)
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=1,               # log & save weights each logging_steps
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    learning_rate=2e-5,
    save_strategy='epoch',
    save_total_limit=3,
    metric_for_best_model='f1'
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

In [ ]:
# trainer.evaluate()

In [ ]:
model.save_pretrained('./models')